In [2]:
print(sys.executable)

/mnt/d/BeCode/BXL-Bouman-2.22/Content/06.Deep_learning/03-NLP/KPMG-Usecase/.env/bin/python


In [3]:
#import json
import csv
import pandas as pd
import re
import os

In [4]:
#This might be unnecessarily long but creates another column for the filenames:

main_folder = 'selection_2'

def get_filename(path):
    filenames = []
    files = [i.path for i in os.scandir(path) if i.is_file()]
 
    #"(?i)FR"
    for filename in files:
        filename = os.path.basename(filename)
        if "_FR" in filename:
            filenames.append(filename)
    return filenames

files = get_filename(main_folder)

with open('only_french_texts.csv', 'w',  encoding = 'utf8', newline = '') as csv_file:
    for _file in files:

        file_name = _file
        with open(main_folder +'/'+ _file,'r') as f:
            text = f.read()

            writer = csv.writer(csv_file)
            writer.writerow([file_name, text])


In [5]:
#csv to dataframe
df = pd.read_csv('french_texts.csv', names = ['filename', 'texts'])

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520 entries, 0 to 519
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  520 non-null    object
 1   texts     520 non-null    object
dtypes: object(2)
memory usage: 8.2+ KB


In [7]:
pd.reset_option("display.max_colwidth")
df.head()

,filename,texts
0,111-111-2018-001734_FR.txt,Neerlegging-Dépôt: 21/02/2018 Regist.-Enregist...
1,111-111-2018-001735_FR.txt,"Service public fédéral Emploi, Travail et Conc..."
2,111-111-2018-002986_FR.txt,PC 111 — CAO individueel opleidingsrecht / CP ...
3,111-111-2018-008643_FR.txt,Neerlegging-Dépôt: 29/05/2018 Regist.-Enregist...
4,111-111-2018-008644_FR.txt,"Service public fédéral Emploi, Travail et\n\nC..."


In [8]:
#split sentences/paragraphs in the text
def split_func(text):
    texts_lst = []
    sents = text.split("\n\n")
    for sent in sents:
        texts_lst.append(sent)
    return texts_lst

In [9]:
df['texts_lst'] = df['texts'].apply(split_func)
df.head()

,filename,texts,texts_lst
0,111-111-2018-001734_FR.txt,Neerlegging-Dépôt: 21/02/2018 Regist.-Enregist...,[Neerlegging-Dépôt: 21/02/2018 Regist.-Enregis...
1,111-111-2018-001735_FR.txt,"Service public fédéral Emploi, Travail et Conc...","[Service public fédéral Emploi, Travail et Con..."
2,111-111-2018-002986_FR.txt,PC 111 — CAO individueel opleidingsrecht / CP ...,[PC 111 — CAO individueel opleidingsrecht / CP...
3,111-111-2018-008643_FR.txt,Neerlegging-Dépôt: 29/05/2018 Regist.-Enregist...,[Neerlegging-Dépôt: 29/05/2018 Regist.-Enregis...
4,111-111-2018-008644_FR.txt,"Service public fédéral Emploi, Travail et\n\nC...","[Service public fédéral Emploi, Travail et, Co..."


In [10]:
def extract_scope(lst):
    scope_lst = []
    for item in lst:
        #pattern1 = r".*\b(convention collective de travail s'applique | Champ d'application)\b.*"
        #pattern2 = r".*\b(La présente convention collective de travail\s*\S* [a-z]'appli\w*)\b.*"
        #pattern3 = r".*\b(convention collective de travail|CCT)\s*\S*[a-z]('appli\w*)\b.*"
        #pattern4 = r".*\b(convention collective de travail|CCT)\s*\S*.*('?appli\w*)\b.*"
        pattern = r".*\b(convention collective de.*travail|CCT).*('*appli\w*)\b.*"
        if re.search(pattern, item, flags=re.IGNORECASE):
            scope_lst.append(item)
        else:
            pass
    return scope_lst

In [11]:
df['scope_of_application'] = df['texts_lst'].apply(extract_scope)
df

,filename,texts,texts_lst,scope_of_application
0,111-111-2018-001734_FR.txt,Neerlegging-Dépôt: 21/02/2018 Regist.-Enregist...,[Neerlegging-Dépôt: 21/02/2018 Regist.-Enregis...,[a présente convention collective de travail s...
1,111-111-2018-001735_FR.txt,"Service public fédéral Emploi, Travail et Conc...","[Service public fédéral Emploi, Travail et Con...",[La présente convention collective de travail ...
2,111-111-2018-002986_FR.txt,PC 111 — CAO individueel opleidingsrecht / CP ...,[PC 111 — CAO individueel opleidingsrecht / CP...,[La présente convention collective de travail ...
3,111-111-2018-008643_FR.txt,Neerlegging-Dépôt: 29/05/2018 Regist.-Enregist...,[Neerlegging-Dépôt: 29/05/2018 Regist.-Enregis...,[Article 1. Cette convention collective de tra...
4,111-111-2018-008644_FR.txt,"Service public fédéral Emploi, Travail et\n\nC...","[Service public fédéral Emploi, Travail et, Co...",[Article 1er. La présente convention collectiv...
...,...,...,...,...
515,311-311-2019-013041_FR.txt,14\n\nNeerlegging-Dépôt: 11/10/2019 | Regist.-...,"[14, Neerlegging-Dépôt: 11/10/2019 | Regist.-E...",[Article 1 - 8 1. La présente convention colle...
516,311-311-2019-013042_FR.txt,Neerlegging-Dépôt: 11/10/2019 Regist.-Enregist...,[Neerlegging-Dépôt: 11/10/2019 Regist.-Enregis...,[Article 1-81. La présente convention collecti...
517,311-311-2020-002305_FR.txt,Neerlegging-Dépôt: 26/02/2020 | Regist.-Enregi...,[Neerlegging-Dépôt: 26/02/2020 | Regist.-Enreg...,[Article 1 - 8 1. La présente convention colle...
518,311-311-2020-002306_FR.txt,)\n\n5\n\nNeerlegging-Dépôt: 26/02/2020\n\nReg...,"[), 5, Neerlegging-Dépôt: 26/02/2020, Regist.-...",[Article 1 - La présente convention collective...
